In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
import os 
import glob
from pathlib import Path
from PIL import Image

In [ ]:
from math import floor as lb

In [ ]:
import cv2

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
import tensorflow as tf

In [ ]:
import keras

In [ ]:
from torchvision import transforms as transforms

In [ ]:
from tensorflow.keras.initializers import random_uniform, glorot_uniform, constant, identity
from tensorflow.keras.layers import Dropout, Input, Add, Dense, Activation, BatchNormalization, Flatten, Conv2D, MaxPooling2D, GlobalMaxPooling2D,ZeroPadding2D
from tensorflow.keras.models import Model, load_model
from keras.models import Sequential

In [ ]:
train_paths = "../input/overnight-hackathon/Train_Images/Train_Images/"
test_paths = "../input/overnight-hackathon/Test_Image/Test_Image/"

In [ ]:
age = []
gender = []
image_path = []
paths_tr = os.listdir(train_paths)
for path in paths_tr:
    image_path.append(path)
    gender.append(str(path.split("_")[3]))
    age.append((int((path.split("_")[4]).split(".")[0])))
    
    
collect  = list(zip(image_path,gender,age))
train_df = pd.DataFrame(collect)
train_df.head()

In [ ]:
train_df["img_path"] = train_df[0]
train_df["gen"] = train_df[1]
train_df["age"] = train_df[2]
train_df.drop([0,1,2],axis=1,inplace=True)


In [ ]:
train_df.head()

In [ ]:
## NOT NEEDED

# gender = []
# image_path = []
# paths_tr = os.listdir(test_paths)
# for path in paths_tr:
#     image_path.append(path)
#     gender.append(str(path.split("_")[3].split(".")[0]))
    
    
# collect_test  = list(zip(image_path,gender))
# test_df = pd.DataFrame(collect_test)
# test_df.head()

In [ ]:
# test_df["img_file"] = test_df[0]
# test_df["gen"] = test_df[1]
# test_df.drop([0,1],axis=1,inplace=True)

In [ ]:

# test_df.head()

In [ ]:
test_csv = pd.read_csv("../input/overnight-hackathon/Test_Data.csv")
test_df = pd.DataFrame(test_csv)

In [ ]:
test_df.head()

In [ ]:
for i in range(len(test_df)):
    test_df.loc[i,"gen"] = ((test_df.loc[i,"img_file"]).split("_")[3]).split(".")[0]

In [ ]:
test_df.head()

In [ ]:
img = Image.open(test_paths+test_df.img_file[1])
plt.imshow(img)


In [ ]:
train_df.isnull().any()

In [ ]:
test_df.isnull().any()

In [ ]:
print(train_df.dtypes,test_df.dtypes)

In [ ]:
print(test_df.gen.unique())

In [ ]:
dummy = pd.get_dummies(train_df["gen"])
train_df = pd.merge(left = train_df,right =dummy,left_index = True,right_index = True)
train_df.drop("gen",axis=1,inplace =True)

In [ ]:
dummy_test = pd.get_dummies(test_df["gen"])
test_df = pd.merge(left = test_df,right = dummy_test,left_index =True,right_index = True)
test_df.drop("gen",axis=1,inplace =True)

In [ ]:
test_df.head()

In [ ]:
test_df.head()

In [ ]:
sns.distplot(train_df.age)

In [ ]:
train ,val = train_test_split(train_df , train_size =0.8 )

In [ ]:
train.head()

In [ ]:
print(train.shape,val.shape)

In [ ]:

dim = (128,128)
x_train, x_test = [], []
for file in train.img_path:
    img = cv2.imread(train_paths +file)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    img = np.array(img)
    x_train.append(img)

# x_train = np.array(x_train)

# x_train = x_train.reshape(len(x_train), 128,128,1)


# x_val = np.array(x_val)

# x_test = x_test.reshape(len(x_test),128,128,1)




In [ ]:

for file in val.img_path:
    img = cv2.imread(train_paths +file)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    img = np.array(img)
    x_test.append(img)

In [ ]:
li = np.array(x_train)

In [ ]:
li2 = np.array(x_test)

In [ ]:
li2.shape

In [ ]:
age_train = np.array(train.age)
age_test = np.array(val.age)

In [ ]:
ip_size = (128,128,3)

In [ ]:
inputs = Input((ip_size))

In [ ]:


model = Sequential()

pretrained_model= tf.keras.applications.ResNet50(include_top=False,
                  input_shape=(128,128,3),
                  pooling='avg',classes=None,
                  weights=None)
for layer in pretrained_model.layers:
       layer.trainable=False

model.add(pretrained_model)



In [ ]:
model.compile(optimizer = 'adam', loss =['mse'])
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='relu'))

In [ ]:
X = Conv2D(64, (3, 3), activation='relu', kernel_initializer = glorot_uniform(seed=0))(inputs)
X = BatchNormalization(axis = 3)(X)
X = MaxPooling2D((3, 3))(X)

X = Conv2D(128, (3, 3), activation='relu')(X)
X = MaxPooling2D((2, 2), strides=(2, 2))(X)

X = Conv2D(256, (3, 3), activation='relu')(X)
X = MaxPooling2D((2, 2))(X)

X = Flatten()(X)

dense_2 = Dense(256, activation='relu' )(X)
dense_3 = Dense(128, activation='relu' )(dense_2)

dropout_2 = Dropout(0.4)(dense_3)

output_2 = Dense(1, activation='relu', name='age_output')(dropout_2)

model = Model(inputs=[inputs], outputs=[output_2])

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer = 'Adam',loss = 'mse')
history = model.fit(x=li,y=age_train,validation_split=0.2,epochs = 16, batch_size=128,shuffle = True)

In [ ]:
index=1438
print("Original: Age = " ,age_test[index])

pred = model.predict(li2[index].reshape(1,128, 128, 3))

print(pred.mean())

In [ ]:
X_pred = []
for file in test_df.img_file:
    t_img = cv2.imread(test_paths+file)
    t_img = cv2.cvtColor(t_img,cv2.COLOR_BGR2RGB)
    t_img = cv2.resize(t_img,dim, interpolation = cv2.INTER_AREA)
    X_pred.append(np.array(t_img))

In [ ]:
li3 = np.array(X_pred)

In [ ]:
li3.shape

In [ ]:
Y_pred = []
for index in range(len(X_pred)):
    pred=model.predict(X_pred[index].reshape(1,128,128,3))
    pred=lb(pred.mean())
    Y_pred.append(pred)

In [ ]:
new_df = test_df.drop(["F","M"],axis=1)

In [ ]:
new_df.head()

In [ ]:
img = cv2.imread(test_paths+new_df.img_file[1])

In [ ]:
plt.imshow(img)

In [ ]:
new_df["age"] = Y_pred

In [ ]:
new_df.head()

In [ ]:
new_df.to_csv("atlast.csv",index = False)

In [ ]:
new_df.shape